In [0]:
# Databricks restarts the kernal after every pip install call, so we have to install everything at once
%pip install pymongo[srv] wordcloud vaderSentiment torch transformers

Python interpreter will be restarted.
Collecting pymongo[srv]
 Downloading pymongo-4.3.3-cp37-cp37m-manylinux2014_x86_64.whl (501 kB)
Collecting wordcloud
 Downloading wordcloud-1.8.2.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
Collecting vaderSentiment
 Downloading vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)
Collecting torch
 Downloading torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
Collecting transformers
 Downloading transformers-4.26.1-py3-none-any.whl (6.3 MB)
Collecting dnspython<3.0.0,>=1.16.0
 Downloading dnspython-2.3.0-py3-none-any.whl (283 kB)
Collecting pillow
 Downloading Pillow-9.4.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Requirement already satisfied: matplotlib in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (3.1.3)
Requirement already satisfied: numpy>=1.6.1 in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (1.18.1)
Requirement already satisfied: requests in /databricks/python3/lib/python3.7/site-packages (from vaderSentiment) (2.22.0)
Collecting nvidia-cuda-runtime-cu11==11.7.99; platform_system == "Linux"
 Downloading nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
Collecting nvidia-cudnn-cu11==8.5.0.96; platform_system == "Linux"
 Downloading nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
Collecting typing-extensions
 Downloading typing_extensions-4.5.0-py3-none-any.whl (27 kB)
Collecting nvidia-cuda-nvrtc-cu11==11.7.99; platform_system == "Linux"
 Downloading nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
Collecting nvidia-cublas-cu11==11.10.3.66; platform_system == "Linux"
 Downloading nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
Collecting pyyaml>=5.1
 Downloading PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
Collecting huggingface-hub<1.0,>=0.11.0
 Downloading huggingface_hub-0.13.1-py3-none-any.whl (199 kB)
Collecting filelock
 Downloading filelock-3.9.0-py3-none-any.whl (9.7 kB)
Collecting regex!=2019.12.17
 Downloading regex-2022.10.31-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (757 kB)
Collecting packaging>=20.0
 Downloading packaging-23.0-py3-none-any.whl (42 kB)
Collecting tqdm>=4.27
 Downloading tqdm-4.65.0-py3-none-any.whl (77 kB)
Collecting tokenizers!=0.11.3,<0.14,>=0.11.1
 Downloading tokenizers-0.13.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
Collecting importlib-metadata; python_version < "3.8"
 Downloading importlib_metadata-6.0.0-py3-none-any.whl (21 kB)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.4.6)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.8.1)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (1.1.0)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (0.10.0)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests->vaderSentiment) (1.25.8)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests->vaderSentiment) (2.8)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests->vaderSentiment) (2020.6.20)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests->vaderSentiment) (3.0.4)
Requirement already satisfied: wheel in /databricks/python3/lib/python3.7/site-packages (from nvidia-cuda-runtime-cu11==11.7.99; platform_system == "Linux"->torch) (0.34.2)
Requirement already satisfied:

In [0]:
import numpy as np
import pandas as pd
import pymongo

import pyspark
from pyspark.sql import Row
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, StructType, StructField

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import transformers
from transformers import pipeline

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [0]:
username = "admin"
password = "goal-diggers"
database = "reviewdb"
collection_name = "amazon"
ip_address = "msds697-goal-diggers.5f8jw.mongodb.net"
client = pymongo.MongoClient(f"mongodb+srv://{username}:{password}@{ip_address}")
db = client[database]
amazon_collection = db[collection_name]
amazon_collection.find_one()

Out[2]: {'_id': ObjectId('63fc65edf6b47c884faf1386'),
 'ASIN': 'B00015VKT4',
 'reviews': [{'review': "My first review but I have to say......About 15 years ago I developed an allergy to nickle (earrings, necklaces, belt buckles, buttons, bracelets!) and stopped wearing most jewelry. I have to tape over my belt buckles or any metal that contacts my skin and was diagnosed as severely allergic by my doc after allergy skin test. About a year ago I got my ears repierced with my daughter so we could have fun wearing jewelry and have tried every kind of hypo allergenic earring I could find: Studex, Sterling Silver 925, Surgical Steel, 14k gold, and had allergic reactions to all of them if worn for even an hour. Studex that my ears were pierced with was the worst! Bleeding, red itchy swollen earlobes a few weeks to heal in between each time I wore earrings! Searched Amazon for an hour to find these and did some research. They work as promised! I had to force them through (holes had partially closed) and had some initial healing for about 2 weeks. Now I'm wearing them 24/7 no problems! It's a first and really wonderful. I will get more. This size is a bit small for a main earring so I will get the next size up. THANK YOU.",
 'rating': 5.0,
 'title': 'No irritation for severe nickle allergy sufferer!'},
 {'review': 'It is refreshing to wear these earrings because they don\'t make my earlobes swell up. My ears are too sensitive for "sensitive" earrings and I was about ready to give up and resign myself to clip earrings. Fortunately I found these. They are more expensive than other earrings due to the metal, but well worth it. It\'s like my ears don\'t even know they\'re there :) .',
 'rating': 5.0,
 'title': 'a relief to wear these'},
 {'review': "I just had my ears pierced a little over two months ago and I thought the minor discomfort was part of the healing process. I decided to try these but honestly didn't expect there to be a difference. Within hours of wearing these my ears had no discomfort at all. Again, my ears weren't red or swollen but just had a very minor ache. I'd recommend these to people even if it's not the last resort to wearing earrings.\n\nI also really like the small simple style so these were perfect for me anyway. I've found it very difficult to find earrings I like in stores because they all seem so big and overbearing. I'm sure that I will end up buying more pairs of these in a similar style and definitely give preference to titanium earrings.",
 'rating': 5.0,
 'title': 'Pleasantly Surprised'},
 {'review': "I just got these a few weeks ago (along with a pair of titanium hoops) and love them. I haven't been able to wear earrings for years because of nickel allergies, and these earrings haven't given me any trouble whatsoever!",
 'rating': 5.0,
 'title': 'I can wear earrings again!'},
 {'review': 'these are the only earrings that have not caused an allergic reaction and they are very pretty too.',
 'rating': 5.0,
 'title': 'lovely'},
 {'review': 'I have only owned these earrings a few weeks, but I\'m quite pleased with them. Before buying these, I had been completely unable to wear earrings for many years. My earlobes would swell, turn red, and bleed immediately after inserting even "hypoallergenic" earrings. I\'ve tried other brands, such as Simply Whispers, which I really wanted to love, but I simply couldn\'t wear them. I\'d given up and given away most of my earrings and hadn\'t bought or tried any earrings for a few years.\n\nWhen these first arrived, I had trouble inserting them because my earlobes had almost grown closed, and I traumatized my ears a bit. Over the next several days, I removed them at night, then irritated my ears again when I reinserted them in the morning. Finally, I decided to just leave them in and allow my ears to heal, and that has worked just fine. Now my ears have healed, and I haven\'t bothered to remove the earrings for a couple of weeks.\n\nI noticed that in a review for one of th

In [0]:
# Akul
pipeline = [
    { "$match": { "ASIN": "B00015VKT4" } },
    { "$unwind": "$reviews" },
    { "$group": { "_id": "$reviews.rating", "count": { "$sum": 1 } } }
]

projection = {
    "ASIN": 1,
    "reviews": 1
             }
      
pipeline = [
    {"$unwind": "$reviews"},
    {"$project": projection}
]

results = amazon_collection.aggregate(pipeline)



data = []    
for doc in results:
    _id = doc['_id']
    ASIN = doc['ASIN']
    reviews = doc['reviews']['review']
    rating = doc['reviews']['rating']
    title = doc['reviews']['title']
    data.append([_id, ASIN, reviews, rating, title])

df = pd.DataFrame(data, columns=['_id', 'ASIN', 'reviews', 'rating', 'title'])
df = df.head(95000)

In [0]:
# Define the threshold for a "good" review
threshold = 4

# Convert the reviews and title columns into bag-of-words features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['reviews'] + ' ' + df['title'])

# Convert the sparse matrix X to a dense numpy array
X = X.toarray()

# Create the target variable
y = df['rating'].apply(lambda x: 1 if x >= threshold else 0)

# Convert the pandas series y to a numpy array
y = y.values

# Define the PyTorch dataset and dataloader
class ReviewDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = ReviewDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the logistic regression model using PyTorch
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionModel(X.shape[1])

# Define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

# Train the model using PyTorch
for epoch in range(5):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d Loss: %.3f' % (epoch + 1, running_loss / len(dataloader)))

# Evaluate the model
with torch.no_grad():
    inputs = torch.tensor(X, dtype=torch.float32)
    labels = torch.tensor(y, dtype=torch.float32)
    outputs = model(inputs)
    predictions = outputs.squeeze().cpu().numpy()
    mae = np.mean(np.abs(predictions - y))
    print('MAE:', mae)

Epoch 1 Loss: 0.207
Epoch 2 Loss: 0.111
Epoch 3 Loss: 0.091
Epoch 4 Loss: 0.082
Epoch 5 Loss: 0.077
MAE: 0.07339412879419072

In [0]:
# Define the input example
input_example = vectorizer.transform(["this item is a piece of junk, I hate it"]).toarray()

# Convert the input example to a PyTorch tensor
input_tensor = torch.tensor(input_example, dtype=torch.float32)

# Make the prediction using the trained model
with torch.no_grad():
    output_tensor = model(input_tensor)
    prediction = output_tensor.item()

# Print the prediction
print("Prediction:", prediction)

Prediction: 0.09791531413793564

In [0]:
# Define the input example
input_example = vectorizer.transform(["Great! I love it!"]).toarray()

# Convert the input example to a PyTorch tensor
input_tensor = torch.tensor(input_example, dtype=torch.float32)

# Make the prediction using the trained model
with torch.no_grad():
    output_tensor = model(input_tensor)
    prediction = output_tensor.item()

# Print the prediction
print("Prediction:", prediction)

Prediction: 0.9960452914237976

In [0]:
# Patricia
# https://spacy.io/universe/project/eng_spacysentiment
!pip install eng-spacysentiment
import eng_spacysentiment
nlp = eng_spacysentiment.load()
text = "Welcome to Arsenals official YouTube channel Watch as we take you closer and show you the personality of the club"
doc = nlp(text)
print(doc.cats)
rating_pred = doc.cats['positive']
print(rating_pred)
# label = amazon_review/5
# error = rating_pred - amazon_label




{'positive': 0.9402117729187012, 'negative': 0.05978821963071823}
0.9402117729187012

In [0]:
# Patricia
row = 8
text = df['reviews'][row]
print(text)
doc = nlp(text)
rating_pred = doc.cats['positive']
print(rating_pred)
label = df['rating'][row]/5
print(label)
error = rating_pred - label
print(error)

They are really nice for the price! Perfect, made out of titanium for those who have allergies to the metals used in mall piercing places!
0.9996743202209473
1.0
-0.0003256797790527344

In [0]:
# Patricia
df['rating_perc'] = df['rating']/5
df['review_nlp'] = df['reviews'].apply(nlp)
df['spacy_prediction'] = df['review_nlp'].apply(lambda row : row.cats['positive'])
df['spacy_error'] = abs(df['rating_perc']-df['spacy_prediction'])
mae = df['spacy_error'].mean()
mae # 0.31524060 

Out[9]: 0.3153731920527474

In [0]:
df[df['rating_perc']!= 1].head(4)

In [0]:
# VMK

from transformers import pipeline
# sentiment_pipeline = pipeline("sentiment-analysis")
# sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
sentiment_pipeline = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

Downloading (…)lve/main/config.json: 0%| | 0.00/687 [00:00<?, ?B/s]Downloading (…)lve/main/config.json: 100%|██████████| 687/687 [00:00<00:00, 225kB/s]
Downloading pytorch_model.bin: 0%| | 0.00/1.42G [00:00<?, ?B/s]Downloading pytorch_model.bin: 1%|▏ | 21.0M/1.42G [00:00<00:09, 149MB/s]Downloading pytorch_model.bin: 3%|▎ | 41.9M/1.42G [00:00<00:13, 99.2MB/s]Downloading pytorch_model.bin: 4%|▍ | 62.9M/1.42G [00:00<00:14, 93.3MB/s]Downloading pytorch_model.bin: 6%|▌ | 83.9M/1.42G [00:00<00:14, 90.8MB/s]Downloading pytorch_model.bin: 7%|▋ | 105M/1.42G [00:01<00:12, 109MB/s] Downloading pytorch_model.bin: 9%|▉ | 126M/1.42G [00:01<00:10, 120MB/s]Downloading pytorch_model.bin: 10%|█ | 147M/1.42G [00:01<00:09, 131MB/s]Downloading pytorch_model.bin: 12%|█▏ | 168M/1.42G [00:01<00:11, 106MB/s]Downloading pytorch_model.bin: 13%|█▎ | 189M/1.42G [00:01<00:12, 103MB/s]Downloading pytorch_model.bin: 15%|█▍ | 210M/1.42G [00:01<00:11, 107MB/s]Downloading pytorch_model.bin: 16%|█▌ | 231M/1.42G [00:02<00:11, 100MB/s]Downloading pytorch_model.bin: 18%|█▊ | 252M/1.42G [00:02<00:11, 104MB/s]Downloading pytorch_model.bin: 19%|█▉ | 273M/1.42G [00:02<00:11, 99.0MB/s]Downloading pytorch_model.bin: 20%|█▉ | 283M/1.42G [00:02<00:11, 97.1MB/s]Downloading pytorch_model.bin: 21%|██ | 294M/1.42G [00:02<00:13, 83.7MB/s]Downloading pytorch_model.bin: 22%|██▏ | 315M/1.42G [00:03<00:11, 93.5MB/s]Downloading pytorch_model.bin: 23%|██▎ | 325M/1.42G [00:03<00:12, 89.4MB/s]Downloading pytorch_model.bin: 24%|██▎ | 336M/1.42G [00:03<00:11, 91.7MB/s]Downloading pytorch_model.bin: 25%|██▌ | 357M/1.42G [00:03<00:10, 104MB/s] Downloading pytorch_model.bin: 26%|██▌ | 367M/1.42G [00:03<00:10, 96.3MB/s]Downloading pytorch_model.bin: 27%|██▋ | 377M/1.42G [00:03<00:15, 67.3MB/s]Downloading pytorch_model.bin: 28%|██▊ | 398M/1.42G [00:04<00:11, 90.3MB/s]Downloading pytorch_model.bin: 30%|██▉ | 419M/1.42G [00:04<00:09, 111MB/s] Downloading pytorch_model.bin: 31%|███ | 440M/1.42G [00:04<00:07, 129MB/s]Downloading pytorch_model.bin: 32%|███▏ | 461M/1.42G [00:04<00:06, 140MB/s]Downloading pytorch_model.bin: 34%|███▍ | 482M/1.42G [00:04<00:08, 117MB/s]Downloading pytorch_model.bin: 35%|███▌ | 503M/1.42G [00:04<00:07, 119MB/s]Downloading pytorch_model.bin: 37%|███▋ | 524M/1.42G [00:04<00:06, 135MB/s]Downloading pytorch_model.bin: 38%|███▊ | 545M/1.42G [00:05<00:06, 126MB/s]Downloading pytorch_model.bin: 40%|███▉ | 566M/1.42G [00:05<00:06, 140MB/s]Downloading pytorch_model.bin: 41%|████▏ | 587M/1.42G [00:05<00:06, 139MB/s]Downloading pytorch_model.bin: 43%|████▎ | 608M/1.42G [00:05<00:05, 147MB/s]Downloading pytorch_model.bin: 44%|████▍ | 629M/1.42G [00:05<00:05, 142MB/s]Downloading pytorch_model.bin: 46%|████▌ | 650M/1.42G [00:05<00:05, 146MB/s]Downloading pytorch_model.bin: 47%|████▋ | 671M/1.42G [00:05<00:05, 147MB/s]Downloading pytorch_model.bin: 49%|████▊ | 692M/1.42G [00:06<00:05, 144MB/s]Downloading pytorch_model.bin: 50%|█████ | 713M/1.42G [00:06<00:04, 149MB/s]Downloading pytorch_model.bin: 52%|█████▏ | 734M/1.42G [00:06<00:05, 136MB/s]Downloading pytorch_model.bin: 53%|█████▎ | 755M/1.42G [00:06<00:04, 136MB/s]Downloading pytorch_model.bin: 55%|█████▍ | 776M/1.42G [00:06<00:05, 129MB/s]Downloading pytorch_model.bin: 56%|█████▌ | 797M/1.42G [00:06<00:04, 141MB/s]Downloading pytorch_model.bin: 58%|█████▊ | 818M/1.42G [00:07<00:04, 142MB/s]Downloading pytorch_model.bin: 59%|█████▉ | 839M/1.42G [00:07<00:03, 153MB/s]Downloading pytorch_model.bin: 60%|██████ | 860M/1.42G [00:07<00:03, 158MB/s]Downloading pytorch_model.bin: 62%|██████▏ | 881M/1.42G [00:07<00:04, 131MB/s]Downloading pytorch_model.bin: 63%|██████▎ | 902M/1.42G [00:07<00:04, 129MB/s]Downloading pytorch_model.bin: 66%|██████▌ | 933M/1.42G [00:07<00:03, 149MB/s]Downloading pytorch_model.bin: 68%|██████▊ | 965M/1.42G [00:07<00:02, 164MB/s]Downloading pytorch_model.bin: 69%|██████▉ | 986M/1.42G [00:08<00:02, 163MB/s]Downloading pytorch_model.bin: 71%|███████ | 1.01G/1.42

In [0]:
# VMK

def m_bert(i):
    try:
        i = sentiment_pipeline(str(i))[0]
        s = i['score']/.4
        if i['label'] == 'POSITIVE':
            s += 2.5
        else:
            s = 2.5-s
        return s
    except:
         return 2.5

In [0]:
# VMK

df_s = df.head(10).copy()

df_s['m_prediction'] = df_s['reviews'].apply(m_bert)
df_s['m_error'] = abs(df_s['rating']-df_s['m_prediction'])
mae = df_s['m_error'].mean()

print(df_s.head(2).T)

mae #for the whole dataset: 0.795